In [1]:
#default_exp utils.sftp

In [2]:
#hide
from nbdev.showdoc import *

# SFTP

In [3]:
#export
from fastcore.basics import store_attr
from fastcore.meta import delegates
from sdsde.snowflake.query import SnowflakeConnect
from sdsde.snowflake.copyinto import CopyInto
from sdsde.azure.filehandling import unlink_files
from sdsde.utils.parseyaml import ParseYaml
from sdsde.wrapper.azurewrapper import blob_pusher

import logging
import pysftp
import sys
import pandas as pd
import os

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [4]:
#export


@delegates()
class SFTP:

    def __init__(self, yaml_file: str = None, yaml_file_path: str = None, yaml_section: str = None,
                 logger=None, **kwargs):
        """
        __init__ SFTP sdsde is looking for a yaml file that can be seen in the
        ``02_utils_stfp``, which has more than the needed variables to be be able to
        uttlize this class. This class was created to unifiy the majority of the SFTP
        calls that we have begun to accumulate and rather than have many functions
        that are slightly different having one function that does 80-90% of what we
        need as a sdsde & RDE Team was the goal of this function.

        This function is in current development and will likely change, but the __init__
        checks for proper connection to sftp host as well as gets the yaml variables need
        that are passed through out this function as well as initializes a snowflake
        connection.

        >> Note: This needs a yaml file

        TODO: write function to intake just a python
        dictionary or give it the option. This would take just setting everything to
        None and then giving a dict_only option simple, but don't want to write more
        at the moment.

        TODO: Re-write to have more delegation and work cleaner, the next time this
        is visited this will change a lot to allow for more flexibility. This has been
        proven to work with LIVERAMP as well as MTA for the needs that have decreased
        the amount of development needed even in this state.

        Args:
        * yaml_file (str): Yaml file name
        * yaml_file_path (str): Yaml file path
        * yaml_section (str, optional): The Section is optional, but when giving
        a section you will be able to pull specfic sections of the yaml file say
        if you needed to loop this function and have multi sftp drops in one repo. Defaults to None.
        * logger (optional): logger sdsde staple for most functions will default
        to standard logging. Defaults to None.
        """
        self._logger = logger if logger is not None else logging.getLogger(__name__)
        yaml_file_path = yaml_file_path if yaml_file_path.endswith('/') else yaml_file_path + '/'
        if yaml_section is None:
            self.sftp_yaml = ParseYaml(os.path.join(yaml_file_path, yaml_file)).get_yaml()
        else:
            self.sftp_yaml = ParseYaml(os.path.join(yaml_file_path, yaml_file)).get_yaml([yaml_section])
        self.cnopts = pysftp.CnOpts()
        self.cnopts.hostkeys = None
        try:
            self.connection = pysftp.Connection(host=os.environ.get(self.sftp_yaml['sftp_host']),
                                                username=os.environ.get(self.sftp_yaml['sftp_user']),
                                                password=os.environ.get(self.sftp_yaml['sftp_pass']),
                                                private_key=os.environ.get(self.sftp_yaml['sftp_pass']),
                                                port=self.sftp_yaml['sftp_port'],
                                                cnopts=self.cnopts)
            self._logger.info('SFTP Connection Established...')
        except pysftp.ConnectionException:
            self._logger.error('failed to connect to SFTP')
        self.connection.close()
        self._logger.error('SFTP Test Connection Closed')
        self._logger.info(f"this is a {os.environ['prod_or_dev']} run..")
        self.sf = SnowflakeConnect(sfAccount=os.environ.get(self.sftp_yaml['sfAccount']),
                                sfUser=os.environ.get(self.sftp_yaml['sfUser']),
                                sfPswd=os.environ.get(self.sftp_yaml['sfPswd']),
                                sfWarehouse=os.environ.get(self.sftp_yaml['sfWarehouse']),
                                sfDatabase=os.environ.get(self.sftp_yaml['sfDatabase']),
                                sfSchema=os.environ.get(self.sftp_yaml['sfSchema']),
                                sfRole=os.environ.get(self.sftp_yaml['sfRole']))
        store_attr()

    def sftp_upload(self,
                    upload_dir: str = '.',
                    file_dir: str = '.',
                    azure_blob_dest='.',
                    suffix: str = '',
                    query: bool = True,
                    store: bool = True,
                    overwrite: bool = False):
        """
        SFTP Upload for 80-90% of the needs that could be thought of at
        time of development. This function allows the user to decide if they
        want to be able to query from snowflake -> Azure Blob -> sftp.
        Giving the option to the user, but defaults to option above

        Args:
        * upload_dir (str, optional): upload directory to sftp. Deafults to '.'.
        * file_dir (str, optional): file directory for files needed (SQL). Defaults to '.'.
        * azure_blob_dest(str, optional): for prod of dev in the drop. Defaults to '.'.
        * suffix (str, optional): adding unique suffix to file name. Defaults to ''.
        * query (bool, optional): Query Snowflake to get data. Defaults to False.
        * store (bool, optional): Store in Azure Blob. Defaults to False.
        * overwrite (bool, optional): Overwrite file in blob if same file name. Defaults to False.
        """
        file_name = self.sftp_yaml['sftp_filename'] + suffix + self.sftp_yaml['sftp_filetype']
        if query is True:
            self.snowflake_query(file_name=file_name, file_dir=file_dir)
        if store is True:
            self.azure_load(file_name, azure_blob_dest, overwrite)
        sftpconnection = self.sftp_connection()
        with sftpconnection.cd(upload_dir):
            self._logger.info(f'checking if {file_name} already exisits...')
            if file_name in sftpconnection.listdir(upload_dir):
                self._logger.warning('File Already Exisits And Will Be Over Written')
            try:
                sftpconnection.put(os.path.join(file_name))
                if sftpconnection.exists(f"{file_name}"):
                    self._logger.info(f"{file_name} recieved and dropped to {upload_dir}")
            except AttributeError:
                self._logger.error('file did not get dropped in sftp')
                sys.exit()
        unlink_files([file_name])
        self._logger.info('sftp upload complete')

    def sftp_grab(self,
                  table_name_sf: str = '',
                  prod_or_dev: str = '',
                  file_name: str = '',
                  download_dir: str = '.',
                  suffix: str = '',
                  azure_blob_dest='.',
                  store: bool = True,
                  create_snowflake_table: bool = True,
                  varchar: bool = True,
                  overwrite: bool = False):
        """
        SFTP function that will grab a file from and sftp host and
        has the options to add it to azure blob and snowflake, but
        if they are both set to False then it will be a simple grab
        of a file from an sftp host.

        Args:
        * table_name_sf (str, optional): table name to load into snowflake
        * prod_or_dev: (str, optional): to tell function if it's a dev run or not
        * file_name (str, optional): optional because MTA needs flex. Defaults to None.
        * download_dir (str, optional): upload directory to sftp. Defaults to '.'.
        * suffix (str, optional): [description]. Defaults to ''.
        * azure_blob_dest(str, optional): for prod of dev in the drop. Defaults to '.'.
        * store (bool, optional): [description]. Defaults to True.
        * create_snowflake_table (bool, optional): [description]. Defaults to True.
        * varchar: (bool, optional): this will default all dytpes to varchars if True.
        * overwrite (bool, optional): [description]. Defaults to False.
        """
        if file_name == '':
            file_name = self.sftp_yaml['sftp_filename'] + suffix + self.sftp_yaml['sftp_filetype']
        if suffix != '':
            file_name = self.sftp_yaml['sftp_filename'] + suffix + self.sftp_yaml['sftp_filetype']
        self._logger.info(f"file name {file_name}")
        sftpconnection = self.sftp_connection()
        with sftpconnection.cd(download_dir):
            sftpconnection.get(file_name)
            self._logger.info(f"""{file_name} recieved and dropped to
                               {os.path.join(download_dir,file_name)}""")
        self._logger.info('Beginning Azure and Snowflake push')
        table_name = table_name_sf if prod_or_dev == 'dev' else \
            table_name_sf.lower().replace('dev', os.environ.get(self.sftp_yaml['sfSchema']))
        if store is True:
            self.azure_load(file_name, azure_blob_dest, overwrite)
        if create_snowflake_table is True:
            df = pd.read_csv(file_name, nrows=10000, low_memory=True)
            unlink_files([file_name])
            df.columns = [x.replace(' ', '_') for x in df.columns]
            self.copy_into_snowflake(file_name, azure_blob_dest, table_name_sf, varchar, df)

        self._logger.info(f"""query new table created...
                            {self.sf.run_str_query(f"SELECT TOP 10 * FROM {table_name};").head(2)}
                            """)
        self._logger.info('sftp grab complete')

    def azure_load(self, file_name: str, azure_blob_dest: str, overwrite: bool):
        """
        This function sends the file of interest to the desired
        location in Azure Blob.

        >> Note: when using this with grab and if the file is the
        same the copy into command knows that the meta data
        hasn't change and will not work and will fail.

        Args:
        * file_name (str): file_name to send to azure
        * overwrite (bool): overwrite file in azure
        """
        blob_dest = azure_blob_dest if azure_blob_dest.endswith('/') else azure_blob_dest + '/'
        self._logger.info(f"saving file to azure blob at {blob_dest}")
        blob_pusher(container_name=self.sftp_yaml['container_name'],
                    connection_str=os.environ.get(self.sftp_yaml['connection_string']),
                    file_path=[file_name],
                    blob_dest=[blob_dest],
                    overwrite=overwrite)
        self._logger.info('file to azure load complete')
        self._logger.info('begining sftp upload/download')

    def copy_into_snowflake(self,
                            file_name: str,
                            azure_blob_dest: str,
                            table_name_sf: str,
                            varchar: bool,
                            df: pd.DataFrame):
        """
        Copy into function for SFTP this function will makes sure
        that the table that is being appended to with the copy into
        command exisits and will check if the table exisits in the
        current schema that the snowflake connection is connected to.

        Args:
        * file_name (str): file name in azure blob to append (CopyInto)
        * azure_blob_dest (str): [description]
        * table_name_sf (str): [description]
        * varchar: (bool, optional): this will default all dytpes to varchars if True.
        * df (pd.DataFrame): for when table doesn't exisit will use dytpes
        """
        blob_dest = azure_blob_dest[:-1] if azure_blob_dest.endswith('/') else azure_blob_dest
        blob_dest = blob_dest[1:] if blob_dest.startswith('/') else blob_dest
        ci = CopyInto(sfAccount=os.environ.get(self.sftp_yaml['sfAccount']),
                      sfUser=os.environ.get(self.sftp_yaml['sfUser']),
                      sfPswd=os.environ.get(self.sftp_yaml['sfPswd']),
                      sfWarehouse=os.environ.get(self.sftp_yaml['sfWarehouse']),
                      sfDatabase=os.environ.get(self.sftp_yaml['sfDatabase']),
                      sfSchema=os.environ.get(self.sftp_yaml['sfSchema']),
                      sfRole=os.environ.get(self.sftp_yaml['sfRole']))
        if table_name_sf.split('.')[-1].upper() not in self.sf.run_str_query(f"Show tables in SCHEMA {self.sf.sfDatabase.upper()}.{self.sf.sfSchema.upper()};").name.tolist():
            self._logger.info('new snowflake table being created')
            self.create_sf_table_from_df(df, table_name_sf, varchar)
        else:
            self._logger.warning('Table Already Exist will only appened to current table')
        try:
            self._logger.info('Pushing Top Ranked file to Snowflake')
            ci.insert_csv(blob_name=file_name,
                          blob_path=blob_dest,
                          storage_account=self.sftp_yaml['storage_account'],
                          container_name=self.sftp_yaml['container_name'],
                          table_name=table_name_sf,
                          sas_token=os.environ.get(self.sftp_yaml['insert_token']),
                          encoding=self.sftp_yaml['encoding'],
                          delimiter=self.sftp_yaml['delimiter'],
                          )
        except AssertionError as e:
            self._logger.error(f'snowflake upload failed {e}')
            sys.exit()

    def snowflake_query(self, file_name: str, file_dir: str):
        """
        Function to query and write out query to .csv format
        currently all use cases need this as the consumer and
        vendors send .csv or .csv.gz.

        Args:
        * file_name (str): file name locally to execute
        * file_dir (str): file directory that the sql file is in can be root level
        """
        self._logger.info('beginning sftp query...')
        df = self.sf.execute_file(os.path.join(file_dir, self.sftp_yaml['query_file']))
        self._logger.info(f'Query result shape {df.shape}')
        df.to_csv(file_name, index=False)
        self._logger.info('file written to local root ready for ingestion...')

    def create_sf_table_from_df(self, df: pd.DataFrame, table_name_sf: str, varchar: bool):
        """
        Dynamically create a table from a dataframe and
        change the dtypes to snowflake dytpes this may have
        a limitation, but can be added.

        Args:
        * df (pd.DataFrame): data frame to get dtypes
        * table_name_sf (str): snowflake table name
        * varchar: (bool, optional): this will default all dytpes to varchars if True.
        """
        select_query = f'''
            create or replace table {table_name_sf} (FEATURES_HERE);
            '''
        for k, v in dict(df.dtypes).items():
            select_query = select_query.replace('FEATURES_HERE', f'{k} {self.return_sf_type(str(v), varchar=varchar)}, FEATURES_HERE')
        select_query = select_query.replace(', FEATURES_HERE', '')
        logger.info(select_query)
        self.sf.run_str_query(select_query)

    def return_sf_type(self, dtype: str, varchar: bool):
        """
        simple function to convert dytpes to snowflake dtypes this
        will be come a very useful thing to have as this will dtype
        data from 10,000 rows from a dataframe, and will fail out
        on the push to snowflake. This means the vendor or the current
        SFTP needs to have a cleaning added to it to allow for data
        to make it through the pipeline. TODO: Figure out how to accepet
        a ETL clean of the data from a vendor that can be placed in place

        Args:
        * dtype (str): dtype from a df in sting form
        * varchar (bool): to default all variables to VARCHAR
        this happens due to bad vendor data and can't be resloved
        with out reading in the whole data set with low_memory=False

        Returns:
        * str: snowflake dtype
        """
        if varchar is True:
            dtype = 'VARCHAR'
        elif 'int' in dtype.lower():
            dtype = 'NUMBER'
        elif 'float' in dtype.lower():
            dtype = 'FLOAT'
        elif 'object' in dtype.lower():
            dtype = 'VARCHAR'
        elif 'bool' in dtype.lower():
            dtype = 'VARCHAR'  # TODO: Limitation found before change once resloved by sf
        elif 'date' in dtype.lower():
            dtype = 'DATETIME'  # TODO: Might break with certain datetimes most generic
        else:
            logging.error('odd dtype not seen needs to be resloved...')
            sys.exit()
        return dtype

    def sftp_connection(self):
        """
        SFTP connection due to time out and limitations to pysftp
        we will be using this function in the connection to pysftp
        """
        connection = pysftp.Connection(host=os.environ.get(self.sftp_yaml['sftp_host']),
                                       username=os.environ.get(self.sftp_yaml['sftp_user']),
                                       password=os.environ.get(self.sftp_yaml['sftp_pass']),
                                       private_key=os.environ.get(self.sftp_yaml['sftp_pass']),
                                       port=self.sftp_yaml['sftp_port'],
                                       cnopts=self.cnopts)
        return connection

# How to Use

## SFTP Class

In [5]:
show_doc(SFTP)

<h2 id="SFTP" class="doc_header"><code>class</code> <code>SFTP</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>SFTP</code>(**`yaml_file`**:`str`=*`None`*, **`yaml_file_path`**:`str`=*`None`*, **`yaml_section`**:`str`=*`None`*, **`logger`**=*`None`*)



This file is expecting a yaml file to be created with the necessary information in it to be able to complete a SFTP successfully. The information in the yaml file created below is more than it needs for an upload or a grab, but together it will do all of the abilities. 

> **Note/Warning**: the keys must be the same name but the values can be different names as the function is reading from the yaml file.

### Yaml File Create

In [6]:
from pathlib import Path
import yaml
import os

def yaml_parse(file_name_path: str, yaml_section: str = None) -> dict:
    """
    Adding functionality to ParseYaml from sdsde lib creates a
    simple call to pull parts of the yaml file for each function

    Args:
    * file_name (str): Yaml File Name
    * yaml_section (str): Yaml section to read

    Returns:
        dict: yaml section to parse
    """
    if yaml_section is not None:
        yaml = ParseYaml(file_name_path).get_yaml([
            os.environ.get('prod_or_dev', 'dev'), yaml_section])
    else:
        yaml = ParseYaml(file_name_path).get_yaml([os.environ.get('prod_or_dev', 'dev')])
    return yaml


def write_yaml_file(file_path: str, file_name: str, dictionary: dict):
    with open(Path(file_path, file_name), 'w') as f:
        yaml.dump(dictionary, f)


def snowflake_query(sfAccount: str = os.environ.get('sfAccount', None),
                    sfUser: str = os.environ.get('sfUser', None),
                    sfPswd: str = os.environ.get('sfPswd', None),
                    sfWarehouse: str = os.environ.get('sfWarehouse', None),
                    sfDatabase: str = os.environ.get('sfDatabase', None),
                    sfSchema: str = os.environ.get('sfSchema', None),
                    sfRole: str = os.environ.get('sfRole', None)):
    """Easy Connection To SnowFlake When Environs are set"""
    sf = SnowflakeConnect(sfAccount, sfUser, sfPswd, sfWarehouse,
                       sfDatabase, sfSchema, sfRole)
    return sf


In [8]:
# skip
sftp_dict = dict({
    'sftp': {
        'sfAccount': 'sfAccount',
        'sfUser':  'sfUser',
        'sfPswd':  'sfPswd',
        'sfWarehouse':  'sfWarehouse',
        'sfDatabase':  'sfDatabase',
        'sfSchema':  'sfSchema',
        'sfRole':  'sfRole',
        'sfSchema_Prod' : 'prod',
        'query_file': 'sftp_query.sql',
        'insert_table_name' : 'MACHINELEARNINGOUTPUTS.DEV.SFTP_TEST_TABLE',
        'delimiter' : ',',
        'insert_token' : 'DATALAKE_SAS_TOKEN_SECRET',
        'encoding' : 'UTF-8',
        'storage_account':os.environ['azure_account'],
        'connection_string': 'DATALAKE_CONN_STR_SECRET',
        'blob_dest' : '/sftp_test/grab_test',
        'container_name': 'taladls',
        'sftp_dir_location' : '/uploads/TestNewLRProcess',
        'sftp_host': 'sftp_host',
        'sftp_pass': 'sftp_pass',
        'sftp_user': 'sftp_user',
        'sftp_filename': 'sftp_test',
        'sftp_filetype': '.csv',
        'sftp_port': 22,
        'sftp_key' : 'sftp_key'
             },
})

write_yaml_file('./testing/', 'sftp.yaml', sftp_dict)

### INIT SFTP

In [10]:
show_doc(SFTP)

<h2 id="SFTP" class="doc_header"><code>class</code> <code>SFTP</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>SFTP</code>(**`yaml_file`**:`str`=*`None`*, **`yaml_file_path`**:`str`=*`None`*, **`yaml_section`**:`str`=*`None`*, **`logger`**=*`None`*)



> Make sure you have an SFTP location to play with

In [12]:
# skip
os.environ['sftp_host']=<fill_in_yours>
os.environ['sftp_user']=<fill_in_yours>
os.environ['sftp_pass']=<fill_in_yours

In [13]:
# skip
test = SFTP(yaml_file='sftp.yaml', yaml_file_path='./testing/', yaml_section='sftp')

INFO:paramiko.transport:Connected (version 2.0, client FTP)
INFO:paramiko.transport:Auth banner: b'This system is solely for the use of authorized users for business purposes.\r\nIndividuals using this system are subject to having all of their activities\r\nmonitored and recorded by system personnel. Unauthorized access or use\r\nshall be subject to prosecution.\r\n\r\n'
INFO:paramiko.transport:Authentication (password) successful!
INFO:__main__:SFTP Connection Established...
ERROR:__main__:SFTP Test Connection Closed
INFO:__main__:this is a dev run..
INFO:sdsde.snowflake.query:sqlalchemy snowflake engine created
INFO:sdsde.snowflake.query:connection to snowflake successful


## SFTP Upload

In [9]:
show_doc(SFTP.sftp_upload)

<h4 id="SFTP.sftp_upload" class="doc_header"><code>SFTP.sftp_upload</code><a href="__main__.py#L73" class="source_link" style="float:right">[source]</a></h4>

> <code>SFTP.sftp_upload</code>(**`upload_dir`**:`str`=*`'.'`*, **`file_dir`**:`str`=*`'.'`*, **`azure_blob_dest`**=*`'.'`*, **`suffix`**:`str`=*`''`*, **`query`**:`bool`=*`True`*, **`store`**:`bool`=*`True`*, **`overwrite`**:`bool`=*`False`*)

SFTP Upload for 80-90% of the needs that could be thought of at
time of development. This function allows the user to decide if they
want to be able to query from snowflake -> Azure Blob -> sftp.
Giving the option to the user, but defaults to option above

Args:
* upload_dir (str, optional): upload directory to sftp. Deafults to '.'.
* file_dir (str, optional): file directory for files needed (SQL). Defaults to '.'.
* azure_blob_dest(str, optional): for prod of dev in the drop. Defaults to '.'.
* suffix (str, optional): adding unique suffix to file name. Defaults to ''.
* query (bool, optional): Query Snowflake to get data. Defaults to False.
* store (bool, optional): Store in Azure Blob. Defaults to False.
* overwrite (bool, optional): Overwrite file in blob if same file name. Defaults to False.

In [15]:
# skip
from datetime import datetime
test.sftp_upload(upload_dir=test.sftp_yaml['sftp_dir_location'],
                 file_dir = 'testing/',
                 azure_blob_dest = test.sftp_yaml['blob_dest'],
                 suffix=f"_{datetime.today().strftime('%Y%m%d')}",
                 query=True, store=True, overwrite=True)

INFO:__main__:beginning sftp query...
INFO:sdsde.snowflake.query:query being parsed
INFO:sdsde.snowflake.query:sqlalchemy snowflake engine created
INFO:sdsde.snowflake.query:sql or txt file excecuted
INFO:sdsde.snowflake.query:connection to snowflake has been turned off
INFO:__main__:Query result shape (10, 38)
INFO:__main__:file written to local root ready for ingestion...
INFO:__main__:saving file to azure blob at /sftp_test/grab_test/
INFO:sdsde.azure.filehandling:digitaladls is a valid
INFO:sdsde.azure.filehandling:ContainerAlreadyExists
INFO:sdsde.azure.filehandling:Uploading sftp_test_20220201.csv, to to Azure Storage /sftp_test/grab_test/sftp_test_20220201.csv
INFO:sdsde.azure.filehandling:Azure Upload Complete
INFO:__main__:file to azure load complete
INFO:__main__:begining sftp upload/download
INFO:paramiko.transport:Connected (version 2.0, client FTP)
INFO:paramiko.transport:Auth banner: b'This system is solely for the use of authorized users for business purposes.\r\nIndivid

In [16]:
# skip
from sdsde.azure.filehandling import FileHandling
fh = FileHandling(os.environ.get('DATALAKE_CONN_STR_SECRET'))
check = fh.ls_blob(test.sftp_yaml['container_name'], path=test.sftp_yaml['blob_dest'], recursive=True) 
check = [x.split('/')[-1] for x in check]
assert 'sftp_test' + f"_{datetime.today().strftime('%Y%m%d')}" + '.csv' in check, 'file did not make it to azure blob'

## SFTP Grab

In [10]:
show_doc(SFTP.sftp_grab)

<h4 id="SFTP.sftp_grab" class="doc_header"><code>SFTP.sftp_grab</code><a href="__main__.py#L116" class="source_link" style="float:right">[source]</a></h4>

> <code>SFTP.sftp_grab</code>(**`table_name_sf`**:`str`=*`''`*, **`prod_or_dev`**:`str`=*`''`*, **`file_name`**:`str`=*`''`*, **`download_dir`**:`str`=*`'.'`*, **`suffix`**:`str`=*`''`*, **`azure_blob_dest`**=*`'.'`*, **`store`**:`bool`=*`True`*, **`create_snowflake_table`**:`bool`=*`True`*, **`varchar`**:`bool`=*`True`*, **`overwrite`**:`bool`=*`False`*)

SFTP function that will grab a file from and sftp host and
has the options to add it to azure blob and snowflake, but
if they are both set to False then it will be a simple grab
of a file from an sftp host.

Args:
* table_name_sf (str, optional): table name to load into snowflake
* prod_or_dev: (str, optional): to tell function if it's a dev run or not
* file_name (str, optional): optional because MTA needs flex. Defaults to None.
* download_dir (str, optional): upload directory to sftp. Defaults to '.'.
* suffix (str, optional): [description]. Defaults to ''.
* azure_blob_dest(str, optional): for prod of dev in the drop. Defaults to '.'.
* store (bool, optional): [description]. Defaults to True.
* create_snowflake_table (bool, optional): [description]. Defaults to True.
* varchar: (bool, optional): this will default all dytpes to varchars if True.
* overwrite (bool, optional): [description]. Defaults to False.

In [18]:
# skip
test.sftp_grab(download_dir=test.sftp_yaml['sftp_dir_location'],
               table_name_sf=test.sftp_yaml['insert_table_name'],
               suffix=f"_{datetime.today().strftime('%Y%m%d')}",
               azure_blob_dest= test.sftp_yaml['blob_dest'],
               store=True, create_snowflake_table=True,
               varchar=False)
# Test here is with in the actual function pull --> azure --> to sf check logs this doesn't need a test again

INFO:__main__:file name sftp_test_20220201.csv
INFO:paramiko.transport:Connected (version 2.0, client FTP)
INFO:paramiko.transport:Auth banner: b'This system is solely for the use of authorized users for business purposes.\r\nIndividuals using this system are subject to having all of their activities\r\nmonitored and recorded by system personnel. Unauthorized access or use\r\nshall be subject to prosecution.\r\n\r\n'
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:__main__:sftp_test_20220201.csv recieved and dropped to
                               /uploads/TestNewLRProcess/sftp_test_20220201.csv
INFO:__main__:Beginning Azure and Snowflake push
INFO:__main__:saving file to azure blob at /sftp_test/grab_test/
INFO:sdsde.azure.filehandling:digitaladls is a valid
INFO:sdsde.azure.filehandling:ContainerAlreadyExists
INFO:sdsde.azure.filehandling:Uploading sftp_test_20220201.csv, to to Azure 

# Clean Up

In [19]:
#skip
test.sf.run_str_query('DROP TABLE MACHINELEARNINGOUTPUTS.DEV.SFTP_TEST_TABLE;')

test.connection.close()

INFO:sdsde.snowflake.query:testing connection
INFO:sdsde.snowflake.query:sqlalchemy snowflake engine created
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.
INFO:sdsde.snowflake.query:executing query
INFO:sdsde.snowflake.query:data loaded from snowflake
INFO:sdsde.snowflake.query:connection to snowflake has been turned off
INFO:sdsde.snowflake.query:SFTP_TEST_TABLE successfully dropped.


# Create

In [21]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_azure.ipynb.
Converted 02_utils_dataframes.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 02_utils_stfp.ipynb.
Converted 02_utils_traininghelpers.ipynb.
Converted 02_utils_traininghelpers_fastai.ipynb.
Converted 03_dstools_preparedata.ipynb.
Converted 04_snowflake_copyinto.ipynb.
Converted 04_snowflake_copyinto2.ipynb.
Converted 04_snowflake_query.ipynb.
Converted 05_azure_wrappers.ipynb.
Converted 06_modeling_inference.ipynb.
Converted 06_modeling_inference_fastai.ipynb.
Converted 06_modeling_premodel.ipynb.
Converted 06_modeling_preprocessing.ipynb.
Converted 06_modeling_preprocessing_fastai.ipynb.
Converted 06_modeling_training.ipynb.
Converted 06_modeling_training_fastai.ipynb.
Converted 07_Binary_Classification_Fastai_Example_Notebook.ipynb.
Converted 08_yaml_ingestion_binary_classification.ipynb.
Converted index.ipynb.
